In [1]:
# =====================================
# Cell 1: Setup & Imports
# =====================================
#%pip install transformers datasets accelerate -q
#%pip install sentencepiece -q   # in case needed by certain models

import os
import math
import random
import torch
import pandas as pd
import json
from collections import defaultdict

# Hugging Face
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoModelForCausalLM
)
from datasets import Dataset, DatasetDict


/Users/atakandemirel/Desktop/Project445/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def load_in_scope_data(json_path):

    with open(json_path, "r") as file:
        data = json.load(file)

    def process_in_scope(data_split):
        data_list = []
        for entry in data_split:
            # entry[0] is the query, entry[1] is the intent
            data_list.append({"query": entry[0], "intent": entry[1]})
        return pd.DataFrame(data_list)

    # Extract only the in-scope splits
    # Out of scope data is ignored in data synthesis
    train_df = process_in_scope(data["train"])
    val_df   = process_in_scope(data["val"])
    test_df  = process_in_scope(data["test"])
    
    return train_df, val_df, test_df

# Usage:
train_df, val_df, test_df = load_in_scope_data("data_full.json")

train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))
test_dataset  = Dataset.from_pandas(test_df.reset_index(drop=True))

print("Train shape:", train_dataset.shape)
print("Val shape:",   val_dataset.shape)
print("Test shape:",  test_dataset.shape)


Train shape: (15000, 2)
Val shape: (3000, 2)
Test shape: (4500, 2)


In [ ]:
# Define the pre-trained model name from Hugging Face
model_name_g = "roberta-base"

# Initialize the tokenizer associated with the chosen model
tokenizer_g = AutoTokenizer.from_pretrained(model_name_g)

# Initialize the model for sequence classification
# The `num_labels` parameter is temporarily set to 150 and will be adjusted later based on the dataset
model_g = AutoModelForSequenceClassification.from_pretrained(model_name_g, num_labels=150)


# Gather unique intents from the training set
unique_intents = sorted(list(set(train_dataset["intent"])))
num_labels = len(unique_intents)
print(f"Number of distinct in-scope intents: {num_labels}")

# Adjust the model's classification head if the number of labels has changed
if model_g.config.num_labels != num_labels:
    # Resize the token embeddings in case the tokenizer has been updated
    model_g.resize_token_embeddings(len(tokenizer_g))
    
    # Replace the classifier with a new one that has the correct number of output labels
    # `model_g.config.dim` should be replaced with the correct hidden size if different
    # For RoBERTa, it's typically 768
    model_g.classifier = torch.nn.Linear(model_g.config.hidden_size, num_labels)

# Create mappings from labels to IDs and vice versa
label2id = {label: i for i, label in enumerate(unique_intents)}
id2label = {i: label for label, i in label2id.items()}

# Update the model configuration with these mappings
model_g.config.label2id = label2id
model_g.config.id2label = id2label

def encode_batch_g(batch):
    # Tokenize the 'query' field with truncation and padding
    enc = tokenizer_g(
        batch["query"], 
        truncation=True, 
        padding="max_length", 
        max_length=32  # Maximum sequence length; adjust based on data
    )
    
    # Map each intent label in the batch to its corresponding ID
    enc["labels"] = [label2id[intent] for intent in batch["intent"]]
    
    return enc

# Apply the encoding function to the training and validation datasets
# The `batched=True` parameter processes multiple examples at once for efficiency
# `remove_columns` removes the original columns not needed for training
train_encoded_g = train_dataset.map(encode_batch_g, batched=True, remove_columns=train_dataset.column_names)
val_encoded_g   = val_dataset.map(encode_batch_g,   batched=True, remove_columns=val_dataset.column_names)


# Define training arguments for the Hugging Face Trainer
training_args_g = TrainingArguments(
    output_dir="g_output",                # Directory to save model checkpoints and logs
    eval_strategy="epoch",                # Evaluation is performed at the end of each epoch
    per_device_train_batch_size=8,        # Batch size per device during training
    per_device_eval_batch_size=8,         # Batch size per device during evaluation
    num_train_epochs=2,                    # Total number of training epochs; adjust based on dataset size
    logging_steps=10,                     # Log training metrics every 10 steps
    seed=42                               # Random seed for reproducibility
)

# Initialize the Trainer with the model, training arguments, and datasets
trainer_g = Trainer(
    model=model_g,                        # The sequence classification model to be trained
    args=training_args_g,                 # Training configuration
    train_dataset=train_encoded_g,        # Encoded training dataset
    eval_dataset=val_encoded_g            # Encoded validation dataset
)

# ---------------------------
# Model Training
# ---------------------------

# Start the training process
trainer_g.train()


# Save the fine-tuned model to the specified directory
model_g.save_pretrained("model_g_finetuned")

# Save the tokenizer to the same directory for future use
tokenizer_g.save_pretrained("tokenizer_g_finetuned")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of distinct in-scope intents: 150


  0%|          | 10/3750 [00:04<22:16,  2.80it/s] 

{'loss': 5.0194, 'grad_norm': 5.052273273468018, 'learning_rate': 4.986666666666667e-05, 'epoch': 0.01}


  1%|          | 20/3750 [00:08<21:10,  2.94it/s]

...

100%|██████████| 3750/3750 [22:08<00:00,  2.94it/s]

{'loss': 5.0222, 'grad_norm': 1.918997049331665, 'learning_rate': 0.0, 'epoch': 2.0}


                                                   
100%|██████████| 3750/3750 [22:28<00:00,  2.78it/s]


{'eval_loss': 5.011055946350098, 'eval_runtime': 15.8985, 'eval_samples_per_second': 188.697, 'eval_steps_per_second': 23.587, 'epoch': 2.0}
{'train_runtime': 1348.1063, 'train_samples_per_second': 22.253, 'train_steps_per_second': 2.782, 'train_loss': 5.022470348103841, 'epoch': 2.0}


('tokenizer_g_finetuned/tokenizer_config.json',
 'tokenizer_g_finetuned/special_tokens_map.json',
 'tokenizer_g_finetuned/vocab.json',
 'tokenizer_g_finetuned/merges.txt',
 'tokenizer_g_finetuned/added_tokens.json',
 'tokenizer_g_finetuned/tokenizer.json')

In [4]:
model_name_g_star = "roberta-base"
tokenizer_g_star = AutoTokenizer.from_pretrained(model_name_g_star)
model_g_star = AutoModelForSequenceClassification.from_pretrained(model_name_g_star, num_labels=150)

# Adjust classifier head if needed
if model_g_star.config.num_labels != num_labels:
    model_g_star.resize_token_embeddings(len(tokenizer_g_star))
    model_g_star.classifier = torch.nn.Linear(model_g_star.config.dim, num_labels)

model_g_star.config.label2id = label2id
model_g_star.config.id2label = id2label

def encode_batch_g_star(batch):
    """
    We feed a special token [EMPTY] or just an empty string 
    so the model can't see the real query.
    """
    # We'll feed "[EMPTY]" for each sample
    input_texts = ["[EMPTY]" for _ in batch["query"]]
    enc = tokenizer_g_star(
        input_texts,
        truncation=True, 
        padding="max_length",
        max_length=8
    )
    enc["labels"] = [label2id[intent] for intent in batch["intent"]]
    return enc

train_encoded_g_star = train_dataset.map(encode_batch_g_star, batched=True, remove_columns=train_dataset.column_names)
val_encoded_g_star   = val_dataset.map(encode_batch_g_star,   batched=True, remove_columns=val_dataset.column_names)

training_args_g_star = TrainingArguments(
    output_dir="g_star_output",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_steps=10,
    seed=42
)

trainer_g_star = Trainer(
    model=model_g_star,
    args=training_args_g_star,
    train_dataset=train_encoded_g_star,
    eval_dataset=val_encoded_g_star
)

trainer_g_star.train()

# Save final classifier g^*
model_g_star.save_pretrained("model_gstar_finetuned")
tokenizer_g_star.save_pretrained("tokenizer_gstar_finetuned")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 10/3750 [00:03<17:44,  3.51it/s]

{'loss': 5.0411, 'grad_norm': 4.87498140335083, 'learning_rate': 4.986666666666667e-05, 'epoch': 0.01}


  1%|          | 20/3750 [00:06<17:16,  3.60it/s]

{'loss': 4.999, 'grad_norm': 4.582918167114258, 'learning_rate': 4.973333333333334e-05, 'epoch': 0.01}


  1%|          | 30/3750 [00:09<17:05,  3.63it/s]

...

100%|██████████| 3750/3750 [18:25<00:00,  3.63it/s]

{'loss': 5.0061, 'grad_norm': 2.7860090732574463, 'learning_rate': 0.0, 'epoch': 2.0}


                                                   
100%|██████████| 3750/3750 [18:40<00:00,  3.35it/s]


{'eval_loss': 5.0111284255981445, 'eval_runtime': 7.9533, 'eval_samples_per_second': 377.203, 'eval_steps_per_second': 47.15, 'epoch': 2.0}
{'train_runtime': 1120.6935, 'train_samples_per_second': 26.769, 'train_steps_per_second': 3.346, 'train_loss': 5.022509630330403, 'epoch': 2.0}


('tokenizer_gstar_finetuned/tokenizer_config.json',
 'tokenizer_gstar_finetuned/special_tokens_map.json',
 'tokenizer_gstar_finetuned/vocab.json',
 'tokenizer_gstar_finetuned/merges.txt',
 'tokenizer_gstar_finetuned/added_tokens.json',
 'tokenizer_gstar_finetuned/tokenizer.json')

In [6]:
# Cell 5: Generate Synthetic Data

# Load the GPT-2 medium tokenizer and model for text generation
model_name_opt = "gpt2-medium"
tokenizer_opt = AutoTokenizer.from_pretrained(model_name_opt)
model_opt = AutoModelForCausalLM.from_pretrained(model_name_opt)

# Check and set the available device (MPS, CUDA, or CPU)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    model_opt = model_opt.to(device)
    print("Using MPS device for acceleration.")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    model_opt = model_opt.to(device)
    print("Using CUDA device for acceleration.")
else:
    device = torch.device("cpu")
    model_opt = model_opt.to(device)
    print("Using CPU.")

def build_prompt(intent_name, real_examples, max_examples=3):
    """
    Create a prompt for the language model with example queries.
    """
    random.shuffle(real_examples)
    real_examples = real_examples[:max_examples]
    
    prompt_lines = [
        f"Intent: {intent_name}",
        "",
        "Here are some example user queries for this intent:"
    ]
    for i, ex in enumerate(real_examples, start=1):
        prompt_lines.append(f"{i}) \"{ex}\"")
    prompt_lines.append("")
    prompt_lines.append(f"Now, provide a new user query that also belongs to the '{intent_name}' intent:\n")
    
    return "\n".join(prompt_lines)

def generate_synthetic_utterances(
    prompt_text,
    tokenizer,
    model,
    num_samples=2,
    max_new_tokens=20,
    top_p=0.9,
    temperature=0.9,
    repetition_penalty=1.1
):
    """
    Generate synthetic utterances based on the prompt using the language model.
    """
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

    generated_texts = []
    for _ in range(num_samples):
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                top_p=top_p,
                temperature=temperature,
                repetition_penalty=repetition_penalty,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        gen_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        completion = gen_text[len(prompt_text):].strip()
        generated_texts.append(completion)
    return generated_texts

def generate_synthetic_dataset(
    real_dataset: Dataset,
    tokenizer_llm,
    model_llm,
    data_multiplier=1,
    max_real_examples=3,
    num_gen_per_prompt=1,
    top_p=0.9,
    temperature=0.9,
    repetition_penalty=1.1,
    max_new_tokens=20
):
    """
    Generate a synthetic dataset by augmenting real data with generated queries.
    """
    real_list = real_dataset.to_list()
    # Group queries by their intent
    intent_to_queries = defaultdict(list)
    for row in real_list:
        intent_to_queries[row["intent"]].append(row["query"])

    synthetic_rows = []
    for intent_name, queries in intent_to_queries.items():
        real_count = len(queries)
        target_count = data_multiplier * real_count
        total_generated = 0
        while total_generated < target_count:
            prompt_text = build_prompt(
                intent_name=intent_name,
                real_examples=queries,
                max_examples=max_real_examples
            )
            gen_texts = generate_synthetic_utterances(
                prompt_text=prompt_text,
                tokenizer=tokenizer_llm,
                model=model_llm,
                num_samples=num_gen_per_prompt,
                max_new_tokens=max_new_tokens,
                top_p=top_p,
                temperature=temperature,
                repetition_penalty=repetition_penalty
            )
            for gtext in gen_texts:
                gtext = gtext.strip()
                synthetic_rows.append({
                    "query": gtext,
                    "intent": intent_name,
                    "source": "synthetic"
                })
            total_generated += num_gen_per_prompt
        print(
            f"Finished intent '{intent_name}'. "
            f"Generated {target_count} synthetic samples for it."
        )

    # Convert synthetic data to a Hugging Face Dataset
    df_synth = pd.DataFrame(synthetic_rows)
    synth_dataset = Dataset.from_pandas(df_synth)
    return synth_dataset

# Generate synthetic data with specified parameters
synthetic_dataset = generate_synthetic_dataset(
    real_dataset=train_dataset,
    tokenizer_llm=tokenizer_opt,
    model_llm=model_opt,
    data_multiplier=2,       # Generate ~2x the size of real data -> 30k new data
    max_real_examples=1,     # Use up to 1 real query in prompts
    num_gen_per_prompt=8,    # Generate 8 utterances per prompt
    top_p=0.9,
    temperature=0.9,
    repetition_penalty=1.1,
    max_new_tokens=20
)

# Output the size and a sample of the synthetic dataset
print("Synthetic dataset size:", len(synthetic_dataset))
print("Sample synthetic row:")
print(synthetic_dataset[0])


Using MPS device for acceleration.
Finished intent 'translate'. Generated 200 synthetic samples for it.
Finished intent 'transfer'. Generated 200 synthetic samples for it.
Finished intent 'timer'. Generated 200 synthetic samples for it.
Finished intent 'definition'. Generated 200 synthetic samples for it.
Finished intent 'meaning_of_life'. Generated 200 synthetic samples for it.
Finished intent 'insurance_change'. Generated 200 synthetic samples for it.
Finished intent 'find_phone'. Generated 200 synthetic samples for it.
Finished intent 'travel_alert'. Generated 200 synthetic samples for it.
Finished intent 'pto_request'. Generated 200 synthetic samples for it.
Finished intent 'improve_credit_score'. Generated 200 synthetic samples for it.
Finished intent 'fun_fact'. Generated 200 synthetic samples for it.
Finished intent 'change_language'. Generated 200 synthetic samples for it.
Finished intent 'payday'. Generated 200 synthetic samples for it.
Finished intent 'replacement_card_durati

In [7]:

# PVI Filtering

# Using the trained models g & g^* to compute:
# PVI(x->y) = -log2[g^*(y|empty)] + log2[g(y|x)]

# 1) Probability helpers

def get_label_probs(model, tokenizer, text, use_empty=False):
    """
    Returns a dictionary of label probabilities for the given text or an empty input.
    """
    # Use "[EMPTY]" as input if use_empty is True
    text_input = "[EMPTY]" if use_empty else text
    # Tokenize the input text
    inputs = tokenizer(text_input, return_tensors="pt", truncation=True, padding=True)
    device = torch.device("mps")  # Change to "cuda" or "cpu" if needed
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model = model.to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[0]  # Get logits for the first (and only) example
        probs = torch.softmax(logits, dim=-1)  # Apply softmax to get probabilities
    
    # Map label indices to their probabilities
    label_prob = {id2label[idx]: float(p.item()) for idx, p in enumerate(probs)}
    return label_prob

def compute_pvi_for_sample(query_text, intent_label):
    """
    Computes the PVI value for a single query-intent pair.
    """
    eps = 1e-12  # Small epsilon to prevent log(0)
    # Get probability from model g
    prob_g = get_label_probs(model_g, tokenizer_g, query_text, use_empty=False)
    p_g_xy = prob_g.get(intent_label, 0.0)
    
    # Get probability from model g^*
    prob_g_star = get_label_probs(model_g_star, tokenizer_g_star, query_text, use_empty=True)
    p_g_star_y = prob_g_star.get(intent_label, 0.0)
    
    # Calculate PVI
    pvi_val = -math.log2(max(p_g_star_y, eps)) + math.log2(max(p_g_xy, eps))
    return pvi_val

def filter_synthetic_data_by_pvi(
    synthetic_data: Dataset,
    pvi_threshold=0.5
):
    """
    Filters the synthetic dataset by keeping only samples with PVI >= pvi_threshold.
    Adds a 'pvi' column to the filtered dataset.
    """
    rows = synthetic_data.to_list()
    filtered = []
    
    for row in rows:
        x = row["query"]
        y = row["intent"]
        pvi_val = compute_pvi_for_sample(x, y)
        if pvi_val >= pvi_threshold:
            new_row = dict(row)
            new_row["pvi"] = pvi_val  # Add PVI value to the row
            filtered.append(new_row)
    
    # Convert filtered rows back to a Dataset
    df_filt = pd.DataFrame(filtered)
    return Dataset.from_pandas(df_filt)

# Apply PVI Filtering to Synthetic Data

# This function was not used in the final synthesis
# Initially used, then discarded for average PVI value for each class for filtering.
filtered_synthetic_dataset = filter_synthetic_data_by_pvi(
    synthetic_data=synthetic_dataset,
    pvi_threshold=0.5
)


In [14]:
def compute_classwise_avg_pvi(val_dataset):
    # Store a list of PVI values for each intent
    class_to_pvi_vals = {}
    
    for example in val_dataset:
        x_query = example["query"]
        y_intent = example["intent"]
        
        pvi_val = compute_pvi_for_sample(x_query, y_intent) 
        if y_intent not in class_to_pvi_vals:
            class_to_pvi_vals[y_intent] = []
        class_to_pvi_vals[y_intent].append(pvi_val)
    
    # Compute average per class
    classwise_avg_pvi = {}
    for intent_label, pvi_list in class_to_pvi_vals.items():
        if len(pvi_list) == 0:
            # If somehow a class doesn't have any labels
            classwise_avg_pvi[intent_label] = 0.0
        else:
            avg_val = sum(pvi_list) / len(pvi_list)
            classwise_avg_pvi[intent_label] = avg_val
    
    return classwise_avg_pvi

# Example usage:
threshold_dict = compute_classwise_avg_pvi(val_dataset)

print("Per-class average PVI threshold:")
for k, v in threshold_dict.items():
    print(f"  Class={k}, threshold={v:.4f}")



Per-class average PVI threshold:
  Class=translate, threshold=-0.0256
  Class=transfer, threshold=0.0328
  Class=timer, threshold=0.0395
  Class=definition, threshold=-0.0469
  Class=meaning_of_life, threshold=0.0163
  Class=insurance_change, threshold=0.0612
  Class=find_phone, threshold=-0.0788
  Class=travel_alert, threshold=-0.0040
  Class=pto_request, threshold=-0.0392
  Class=improve_credit_score, threshold=0.1001
  Class=fun_fact, threshold=-0.0602
  Class=change_language, threshold=0.0579
  Class=payday, threshold=-0.1091
  Class=replacement_card_duration, threshold=0.0804
  Class=time, threshold=0.0001
  Class=application_status, threshold=0.0301
  Class=flight_status, threshold=0.0277
  Class=flip_coin, threshold=0.0046
  Class=change_user_name, threshold=-0.0542
  Class=where_are_you_from, threshold=-0.0019
  Class=shopping_list_update, threshold=0.1012
  Class=what_can_i_ask_you, threshold=-0.0076
  Class=maybe, threshold=0.0002
  Class=oil_change_how, threshold=0.0349
  Cl

In [9]:
def filter_synthetic_data_by_pvi_per_class(
    synthetic_data, 
    threshold_dict
):
    """
    For each row (x, y) in synthetic_data, compute PVI(x->y).
    Keep only those with PVI >= threshold_dict[y].
    """
    filtered_rows = []
    original_rows = synthetic_data.to_list()  # if it's a HF Dataset
    for row in original_rows:
        x_query = row["query"]
        y_intent = row["intent"]
        
        pvi_val = compute_pvi_for_sample(x_query, y_intent)
        
        # Compare with the class-specific threshold
        class_threshold = threshold_dict.get(y_intent, 0.0)
        
        if pvi_val >= class_threshold:
            new_row = dict(row)
            new_row["pvi"] = pvi_val
            filtered_rows.append(new_row)
    
    import pandas as pd
    df_filtered = pd.DataFrame(filtered_rows)
    
    from datasets import Dataset
    filtered_dataset = Dataset.from_pandas(df_filtered)
    
    # print stats
    original_count = len(original_rows)
    filtered_count = len(filtered_dataset)
    percent = 100.0 * filtered_count / original_count if original_count > 0 else 0.0
    print(f"Filtered synthetic data: kept {filtered_count}/{original_count} = {percent:.1f}%")
    
    return filtered_dataset

filtered_synth_dataset = filter_synthetic_data_by_pvi_per_class(
    synthetic_data=synthetic_dataset,
    threshold_dict=threshold_dict
)



Filtered synthetic data: kept 13860/30000 = 46.2%


In [ ]:
# Cell 7: Combine real + filtered synthetic

df_real = train_dataset.to_pandas()
df_filt_synth = filtered_synth_dataset.to_pandas()

df_aug = pd.concat([df_real, df_filt_synth], ignore_index=True)
aug_dataset = Dataset.from_pandas(df_aug)
print(f"\nAugmented dataset size: {len(aug_dataset)}")
if len(aug_dataset) > 0:
    print("Augmented sample row:", aug_dataset[0])


Augmented dataset size: 28860
Augmented sample row: {'query': 'what expression would i use to say i love you if i were an italian', 'intent': 'translate', 'source': None, 'pvi': None}


In [11]:
# Define the output file path
output_json_path = "augmented_dataset.json"

# Save the dataset to a JSON file
aug_dataset.to_json(output_json_path)

print(f"\nAugmented dataset has been saved to {output_json_path}")


Creating json from Arrow format: 100%|██████████| 29/29 [00:00<00:00, 596.01ba/s]


Augmented dataset has been saved to augmented_dataset.json


In [12]:
# Write newly generated data to a txt file
df_synthetic = synthetic_dataset.to_pandas()
with open("synthetic_dataset.txt", "w", encoding="utf-8") as file:
    for query in df_synthetic["query"]:
        file.write(query + "\n")

print("Synthetic dataset has been written to 'synthetic_dataset.txt'.")

Synthetic dataset has been written to 'synthetic_dataset.txt'.


In [13]:
print(torch.backends.mps.is_available())  # should be True
print(torch.backends.mps.is_built())

True
True
